In [3]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras
import numpy as np

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
min_loss = 0
max_profit = 0
commission = 5
risk = 0

C:\Users\Bikal\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

In [5]:
symbols = """
select distinct(name) from stockdatamonthly order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [6]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [7]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = None
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        

sql1= """
select distinct(name) from stockdatamonthly order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)

<h1> Updating resources

In [8]:
cursor = len(symbol_list)
percent = cursor // 100


counter = 0
percentage = 0
for symbol in symbol_list:
    if(counter % percent == 0 ):
        print(percentage , "% done")
        percentage += 1
    symbol_data = """
    select * from stockdatamonthly where name = '%s'
    """ % symbol
    counter += 1
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)

0 % done
1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done


In [9]:
sql_dates = """
select distinct(date) from stockdatamonthly order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Trading strategy with the training sets generation

In [10]:
def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global capital
    global min_loss
    global max_profit
    global risk
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdatamonthly where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    
                    
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        buy_stocks(current_price, current_symbol, date)

                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                win += 1
                                current_symbol.flag_for_wins = True
                                risk = risk - 1
                        
                    if current_symbol.stop_order > current_price:                        
                        sell_stocks(current_symbol, current_price, date)
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [11]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def buy_stocks(current_price, current_symbol, date):
    global position
    global remaining_cash
    global risk
    global commission
    
    quantity = (0.02 * remaining_cash) // (current_price - (current_symbol.support * 0.97))
              
    if quantity > 1:
        current_symbol.bought = current_price
        current_symbol.stop_order = current_symbol.support * 0.97
        remaining_cash = remaining_cash - commission - (quantity * current_price)
        position[current_symbol.symbol] = quantity
        print("Buying ", quantity, "stocks of ", current_symbol.symbol, "At price", current_symbol.bought, "on date", date)
        risk += 1 
    else:
        pass

def sell_stocks(current_symbol, current_price, date):
    global position
    global remaining_cash
    global profit
    global commission
    global min_loss
    global max_profit
    global win
    global loss
    global risk
    
    risk = risk - 1
                        
    current_symbol.stop_order = None
    print("Selling ",position[current_symbol.symbol], "stocks of", current_symbol.symbol, " at price", current_price, "on date", date)
    sold = current_price * position[current_symbol.symbol] - commission
    profit_today = sold - current_symbol.bought * position[current_symbol.symbol]

    if profit_today > 0:                           
        if current_symbol.flag_for_wins == False:
            win += 1
        current_symbol.flag_for_wins = False

    else:
        loss += 1                           
        current_symbol.flag_for_wins = False

    if profit_today < min_loss:
        min_loss = profit_today
    if profit_today > max_profit:
        max_profit = profit_today

    profit += profit_today
    remaining_cash = remaining_cash + sold
    del position[current_symbol.symbol]
    current_symbol.bought = None


In [12]:
automatic_trader(symbol_list)

Buying  19.0 stocks of  MTW At price 34.0 on date 2000-04-28
Buying  10.0 stocks of  WSM At price 35.12 on date 2000-04-28
Buying  23.0 stocks of  IBOC At price 43.38 on date 2000-04-28
Buying  20.0 stocks of  LEG At price 22.56 on date 2000-04-28
Buying  103.0 stocks of  SAM At price 8.313 on date 2000-04-28
Buying  7.0 stocks of  TAP At price 55.25 on date 2000-04-28
Buying  29.0 stocks of  GRIF At price 14.0 on date 2000-05-31
Buying  39.0 stocks of  MGEE At price 19.25 on date 2000-05-31
Buying  18.0 stocks of  MOS At price 17.75 on date 2000-05-31
Buying  52.0 stocks of  ICB At price 16.25 on date 2000-06-30
Buying  11.0 stocks of  MRO At price 29.19 on date 2000-06-30
Buying  57.0 stocks of  HIO At price 9.25 on date 2000-06-30
Buying  30.0 stocks of  NXP At price 14.5 on date 2000-06-30
Selling  23.0 stocks of IBOC  at price 32.75 on date 2000-07-31
Buying  18.0 stocks of  OPY At price 19.88 on date 2000-07-31
Buying  3.0 stocks of  BA At price 50.25 on date 2000-07-31
Buying  9

Buying  3.0 stocks of  AAON At price 36.0899 on date 2013-05-31
Selling  3.0 stocks of EEI  at price 11.0 on date 2013-07-31
Selling  41.0 stocks of NBO  at price 12.8901 on date 2013-08-30
Selling  17.0 stocks of MNE  at price 13.24 on date 2013-08-30
Selling  6.0 stocks of BNO  at price 44.9 on date 2013-09-30
Buying  7.0 stocks of  SLY At price 101.54 on date 2013-10-31
Buying  3.0 stocks of  DNB At price 117.69 on date 2013-11-29
Selling  3.0 stocks of THD  at price 67.88 on date 2014-01-31
Buying  3.0 stocks of  INSY At price 59.96 on date 2014-01-31
Buying  23.0 stocks of  EUFN At price 26.1 on date 2014-01-31
Buying  24.0 stocks of  THST At price 6.15 on date 2014-02-28
Buying  3.0 stocks of  WSM At price 67.88 on date 2014-03-31
Selling  39.0 stocks of MGEE  at price 40.7065 on date 2014-03-31
Buying  21.0 stocks of  CUR At price 4.8 on date 2014-03-31
Selling  18.0 stocks of RM  at price 25.07 on date 2014-04-30
Buying  3.0 stocks of  DFJ At price 54.5488 on date 2014-07-31
Se

In [13]:
print("Remaining Cash ::" , remaining_cash)

Remaining Cash :: 1597.963599999999


In [14]:
current_profit = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    remaining_cash += position[key] * stock_equities[key].current_price
    profit += current_profit
    
    if current_profit > 0:
        win += 1
    else:
        loss += 1
    
    if profit > max_profit:
        max_profit = profit
    
    if profit < min_loss:
        min_loss = profit
    
    
print("Maximum Profit is ", max_profit)
print("Minimum Loss is ", min_loss)
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Win Percentage")            
print((win/(loss + win))*100)

Maximum Profit is  6150.789
Minimum Loss is  -261.19999999999993
Profit 6128.539
Remaining cash  15508.538999999997
Win Percentage
54.48275862068965
